Faiss is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning. Faiss is written in C++ with complete wrappers for Python/numpy. Some of the most useful algorithms are implemented on the GPU. It is developed primarily at Meta's Fundamental AI Research group.

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [3]:
loader = TextLoader("TDP.txt")
doc = loader.load()

In [5]:
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 30)
split_doc= text_splitter.split_documents(doc)


Created a chunk of size 1037, which is longer than the specified 1000


In [7]:
embeddings = OllamaEmbeddings( model="gemma2:2b")
db = FAISS.from_documents(split_doc, embeddings)

In [8]:
#querying data
query = "who founded Telugu Desam Party?"
docs=db.similarity_search(query)

In [10]:
docs[0].page_content

'TDP party was founded thinking that we should stand by the people of the state who are struggling with many such problems and put a political party to stand by the people. The party was formed on 29 March 1982 and contested the 1983 elections. The people of Andhra Pradesh, who were fed up with the Congress government which had been ruling for 36 years, gave a great victory to the TDP party in the 1983 elections. On 9 January 1983, NTR was sworn in as the 7th chief minister of Andhra Pradesh.[22]\n\nOn 16 August 1984, when NTR went to America for the operation, Nadendla Bhaskara Rao, one of the MLAs of the TDP party, took the oath of office with the Governor along with his MLAs as the Chief Minister.[23] On 16 September 1984, NTR along with his MLAs went to Delhi and held a march and dharna against the then Prime Minister Indira Gandhi. Then NTR was once again sworn in as the chief minister.'

#### As a retriever
we can also convert a vector store into retriever class. This allows us to easily use it in other lang chain methods, which largely work with retrievers.

In [12]:
retriever = db.as_retriever()
docs = retriever.invoke(query)

In [13]:
docs

[Document(metadata={'source': 'TDP.txt'}, page_content='TDP party was founded thinking that we should stand by the people of the state who are struggling with many such problems and put a political party to stand by the people. The party was formed on 29 March 1982 and contested the 1983 elections. The people of Andhra Pradesh, who were fed up with the Congress government which had been ruling for 36 years, gave a great victory to the TDP party in the 1983 elections. On 9 January 1983, NTR was sworn in as the 7th chief minister of Andhra Pradesh.[22]\n\nOn 16 August 1984, when NTR went to America for the operation, Nadendla Bhaskara Rao, one of the MLAs of the TDP party, took the oath of office with the Governor along with his MLAs as the Chief Minister.[23] On 16 September 1984, NTR along with his MLAs went to Delhi and held a march and dharna against the then Prime Minister Indira Gandhi. Then NTR was once again sworn in as the chief minister.'),
 Document(metadata={'source': 'TDP.tx

#### similarity search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also distance score of query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [14]:
docs_and_score = db.similarity_search_with_score(query)

In [15]:
docs_and_score

[(Document(metadata={'source': 'TDP.txt'}, page_content='TDP party was founded thinking that we should stand by the people of the state who are struggling with many such problems and put a political party to stand by the people. The party was formed on 29 March 1982 and contested the 1983 elections. The people of Andhra Pradesh, who were fed up with the Congress government which had been ruling for 36 years, gave a great victory to the TDP party in the 1983 elections. On 9 January 1983, NTR was sworn in as the 7th chief minister of Andhra Pradesh.[22]\n\nOn 16 August 1984, when NTR went to America for the operation, Nadendla Bhaskara Rao, one of the MLAs of the TDP party, took the oath of office with the Governor along with his MLAs as the Chief Minister.[23] On 16 September 1984, NTR along with his MLAs went to Delhi and held a march and dharna against the then Prime Minister Indira Gandhi. Then NTR was once again sworn in as the chief minister.'),
  25146.453),
 (Document(metadata={'

In [16]:
embedding_vector= embeddings.embed_query(query)

In [18]:
len(embedding_vector)

2304

In [19]:
docs_vector_score=db.similarity_search_by_vector(embedding_vector)

In [20]:
docs_vector_score

[Document(metadata={'source': 'TDP.txt'}, page_content='TDP party was founded thinking that we should stand by the people of the state who are struggling with many such problems and put a political party to stand by the people. The party was formed on 29 March 1982 and contested the 1983 elections. The people of Andhra Pradesh, who were fed up with the Congress government which had been ruling for 36 years, gave a great victory to the TDP party in the 1983 elections. On 9 January 1983, NTR was sworn in as the 7th chief minister of Andhra Pradesh.[22]\n\nOn 16 August 1984, when NTR went to America for the operation, Nadendla Bhaskara Rao, one of the MLAs of the TDP party, took the oath of office with the Governor along with his MLAs as the Chief Minister.[23] On 16 September 1984, NTR along with his MLAs went to Delhi and held a march and dharna against the then Prime Minister Indira Gandhi. Then NTR was once again sworn in as the chief minister.'),
 Document(metadata={'source': 'TDP.tx

In [21]:
#### saving to local and loading
db.save_local("Faiss_index")

In [25]:
#### loadong the saved file
new_db=FAISS.load_local("Faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)

In [26]:
new_similarity_search = new_db.similarity_search_with_score(query)

In [27]:
new_similarity_search

[(Document(metadata={'source': 'TDP.txt'}, page_content='TDP party was founded thinking that we should stand by the people of the state who are struggling with many such problems and put a political party to stand by the people. The party was formed on 29 March 1982 and contested the 1983 elections. The people of Andhra Pradesh, who were fed up with the Congress government which had been ruling for 36 years, gave a great victory to the TDP party in the 1983 elections. On 9 January 1983, NTR was sworn in as the 7th chief minister of Andhra Pradesh.[22]\n\nOn 16 August 1984, when NTR went to America for the operation, Nadendla Bhaskara Rao, one of the MLAs of the TDP party, took the oath of office with the Governor along with his MLAs as the Chief Minister.[23] On 16 September 1984, NTR along with his MLAs went to Delhi and held a march and dharna against the then Prime Minister Indira Gandhi. Then NTR was once again sworn in as the chief minister.'),
  25146.453),
 (Document(metadata={'